Albumentations 라이브러리에서 제공하는 bounding box 형식은 일반적으로 pascal_voc, coco, yolo 등의 형식으로, 이들은 [x_min, y_min, x_max, y_max] 또는 [x, y, width, height] 형식을 사용합니다. 하지만 현재 사용 중인 [[x1, y1, x2, y2, x3, y3, x4, y4]] 형식은 Albumentations에서 바로 지원되지 않으므로, 이를 keypoint로 취급하여 augmentation을 적용해야 합니다.

다음과 같은 방법으로 Albumentations에서 제공하는 keypoint 기능을 이용하여 augmentation을 적용할 수 있습니다.

1. Keypoint 형식으로 변환하기
좌표를 keypoint 형식으로 사용하기 위해, Albumentations의 KeypointParams를 설정합니다. keypoints로 처리된 좌표는 augmentation 중 변환되며, 변환된 좌표를 다시 polygon bounding box 형식으로 사용할 수 있습니다.

2. 코드 예제
다음은 [[x1, y1, x2, y2, x3, y3, x4, y4]] 형식의 좌표를 keypoint로 변환하여 Albumentations로 처리하는 예제 코드입니다.

In [ ]:
import albumentations as A
import cv2

# 이미지 불러오기
image = cv2.imread('receipt.jpg')

# 영수증의 네 꼭짓점 좌표 예시
bbox_coords = [[x1, y1, x2, y2, x3, y3, x4, y4]]
keypoints = [(x1, y1), (x2, y2), (x3, y3), (x4, y4)]

# Augmentation 파이프라인 설정
transform = A.Compose([
    A.RandomRotate90(),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    # 필요한 다른 augmentation 추가 가능
], keypoint_params=A.KeypointParams(format='xy'))

# Augmentation 적용
transformed = transform(image=image, keypoints=keypoints)

# 변환된 이미지와 좌표 가져오기
transformed_image = transformed['image']
transformed_keypoints = transformed['keypoints']

# 변환된 좌표를 bbox 형식으로 다시 변환
transformed_bbox = [int(coord) for point in transformed_keypoints for coord in point]

# 변환된 이미지와 bbox 저장 또는 출력
cv2.imwrite('augmented_receipt.jpg', transformed_image)
print("Transformed BBox Coordinates:", transformed_bbox)

3. 주의사항
keypoint 형식 설정: KeypointParams의 format을 'xy'로 지정하여 각 좌표가 개별 점으로 인식되도록 합니다.
좌표 변환 확인: 변환 후 keypoints로부터 bbox 좌표를 재구성할 때, 각 좌표의 정확성을 확인해야 합니다. 변환 후 좌표를 다시 [[x1, y1, x2, y2, x3, y3, x4, y4]] 형식으로 결합하면 됩니다.
적용된 변환 검토: 이미지를 augmentation한 후, 변환된 bbox가 이미지에 맞는지 시각적으로 확인하는 것도 중요합니다.
이 방법으로 Albumentations에서 기존 bbox 형식을 활용한 augmentation을 수행할 수 있습니다.

In [ ]:
import cv2
import numpy as np
import random
import albumentations as A
from typing import Tuple, List, Optional

class ReceiptAugmentor:
    def __init__(self, image_size: Tuple[int, int] = (800, 1000), random_state: Optional[int] = None):
        self.image_size = image_size
        if random_state is not None:
            random.seed(random_state)
            np.random.seed(random_state)

        # 기본 밝기/대비 변환
        self.base_transform = A.Compose([
            A.OneOf([
                A.RandomBrightness(limit=0.2, p=0.5),
                A.RandomContrast(limit=0.2, p=0.5),
                A.RandomGamma(gamma_limit=(80, 120), p=0.5),
            ], p=0.3),
            A.OneOf([
                A.GaussNoise(var_limit=(10, 50), p=0.3),
                A.MultiplicativeNoise(multiplier=(0.9, 1.1), p=0.3),
            ], p=0.3),
        ])
    
    # 기존 함수는 생략 (이미 있는 _apply_perspective_transform, _apply_rotation, _apply_scale 등)

    def _apply_elastic_transform(self, image: np.ndarray, bboxes: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        transform = A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=1)
        augmented = transform(image=image, keypoints=bboxes.reshape(-1, 2).tolist(), keypoint_params=A.KeypointParams(format='xy'))
        return augmented['image'], np.array(augmented['keypoints']).reshape(-1, 8)
    
    def _apply_grid_distortion(self, image: np.ndarray, bboxes: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        transform = A.GridDistortion(num_steps=5, distort_limit=0.3, p=1)
        augmented = transform(image=image, keypoints=bboxes.reshape(-1, 2).tolist(), keypoint_params=A.KeypointParams(format='xy'))
        return augmented['image'], np.array(augmented['keypoints']).reshape(-1, 8)
    
    def _apply_optical_distortion(self, image: np.ndarray, bboxes: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        transform = A.OpticalDistortion(distort_limit=0.05, shift_limit=0.05, p=1)
        augmented = transform(image=image, keypoints=bboxes.reshape(-1, 2).tolist(), keypoint_params=A.KeypointParams(format='xy'))
        return augmented['image'], np.array(augmented['keypoints']).reshape(-1, 8)
    
    def _apply_motion_blur(self, image: np.ndarray, bboxes: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        transform = A.MotionBlur(blur_limit=7, p=1)
        augmented = transform(image=image, keypoints=bboxes.reshape(-1, 2).tolist(), keypoint_params=A.KeypointParams(format='xy'))
        return augmented['image'], np.array(augmented['keypoints']).reshape(-1, 8)

    def augment(self, image: np.ndarray, bboxes: np.ndarray, augmentation_types: List[str] = None) -> Tuple[np.ndarray, np.ndarray]:
        if augmentation_types is None:
            augmentation_types = ['perspective', 'rotate', 'scale', 'elastic', 'grid', 'optical', 'blur']

        result = self.base_transform(image=image)['image']
        result_bboxes = bboxes.copy()
        
        for aug_type in augmentation_types:
            if aug_type == 'perspective':
                result, result_bboxes = self._apply_perspective_transform(result, result_bboxes)
            elif aug_type == 'rotate':
                result, result_bboxes = self._apply_rotation(result, result_bboxes)
            elif aug_type == 'scale':
                result, result_bboxes = self._apply_scale(result, result_bboxes)
            elif aug_type == 'elastic':
                result, result_bboxes = self._apply_elastic_transform(result, result_bboxes)
            elif aug_type == 'grid':
                result, result_bboxes = self._apply_grid_distortion(result, result_bboxes)
            elif aug_type == 'optical':
                result, result_bboxes = self._apply_optical_distortion(result, result_bboxes)
            elif aug_type == 'blur':
                result, result_bboxes = self._apply_motion_blur(result, result_bboxes)
        
        return result, result_bboxes

In [ ]:
import cv2
import numpy as np
import random
import albumentations as A
from typing import Tuple, List, Optional

class ReceiptAugmentor:
    def __init__(self, image_size: Tuple[int, int] = (800, 1000), random_state: Optional[int] = None):
        self.image_size = image_size
        if random_state is not None:
            random.seed(random_state)
            np.random.seed(random_state)

        # 기본 밝기/대비 변환
        self.base_transform = A.Compose([
            A.OneOf([
                A.RandomBrightness(limit=0.2, p=0.5),
                A.RandomContrast(limit=0.2, p=0.5),
                A.RandomGamma(gamma_limit=(80, 120), p=0.5),
            ], p=0.3),
            A.OneOf([
                A.GaussNoise(var_limit=(10, 50), p=0.3),
                A.MultiplicativeNoise(multiplier=(0.9, 1.1), p=0.3),
            ], p=0.3),
        ])

    def _apply_perspective_transform(self, image: np.ndarray, bboxes: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        transform = A.Perspective(scale=(0.05, 0.1), keep_size=True, p=1)
        augmented = transform(image=image, keypoints=bboxes.reshape(-1, 2).tolist(), keypoint_params=A.KeypointParams(format='xy'))
        return augmented['image'], np.array(augmented['keypoints']).reshape(-1, 8)

    def _apply_rotation(self, image: np.ndarray, bboxes: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        transform = A.Rotate(limit=20, border_mode=cv2.BORDER_CONSTANT, value=(255, 255, 255), p=1)
        augmented = transform(image=image, keypoints=bboxes.reshape(-1, 2).tolist(), keypoint_params=A.KeypointParams(format='xy'))
        return augmented['image'], np.array(augmented['keypoints']).reshape(-1, 8)

    def _apply_scale(self, image: np.ndarray, bboxes: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        transform = A.RandomScale(scale_limit=(0.8, 1.2), interpolation=cv2.INTER_LINEAR, p=1)
        augmented = transform(image=image, keypoints=bboxes.reshape(-1, 2).tolist(), keypoint_params=A.KeypointParams(format='xy'))
        return augmented['image'], np.array(augmented['keypoints']).reshape(-1, 8)

    def augment(self, image: np.ndarray, bboxes: np.ndarray, augmentation_types: List[str] = None) -> Tuple[np.ndarray, np.ndarray]:
        if augmentation_types is None:
            augmentation_types = ['perspective', 'rotate', 'scale', 'elastic', 'grid', 'optical', 'blur']

        result = self.base_transform(image=image)['image']
        result_bboxes = bboxes.copy()
        
        for aug_type in augmentation_types:
            if aug_type == 'perspective':
                result, result_bboxes = self._apply_perspective_transform(result, result_bboxes)
            elif aug_type == 'rotate':
                result, result_bboxes = self._apply_rotation(result, result_bboxes)
            elif aug_type == 'scale':
                result, result_bboxes = self._apply_scale(result, result_bboxes)
            elif aug_type == 'elastic':
                result, result_bboxes = self._apply_elastic_transform(result, result_bboxes)
            elif aug_type == 'grid':
                result, result_bboxes = self._apply_grid_distortion(result, result_bboxes)
            elif aug_type == 'optical':
                result, result_bboxes = self._apply_optical_distortion(result, result_bboxes)
            elif aug_type == 'blur':
                result, result_bboxes = self._apply_motion_blur(result, result_bboxes)
        
        return result, result_bboxes

구겨짐

네, crumpling(구김) 효과를 구현하려면 ElasticTransform 또는 GridDistortion 같은 변환을 활용할 수 있습니다. 이 두 가지 변환은 이미지에 비정형적인 왜곡을 주어 종이가 구겨진 듯한 느낌을 표현할 수 있습니다. 특히 GridDistortion을 사용하면 종이를 구긴 것처럼 불규칙하게 왜곡되는 효과를 줄 수 있으며, Albumentations의 Keypoint 기능을 통해 bbox 좌표도 자동으로 변환됩니다.

구김 효과를 자연스럽게 구현하기 위해서는 텍스처로 추가적인 wrinkling 효과를 넣는 방법도 있습니다. 실제로 그래픽스에서는 wrinkling을 텍스처로 표현하는 경우가 많으며, 이 방법은 주로 쉐이더나 노멀 맵을 통해 표현됩니다. Augmentation만으로는 주름의 깊이와 방향성을 완벽히 표현하기 어렵기 때문에, 이를 위해서는 추가적인 텍스처 레이어를 활용하여 표면의 깊이와 방향감을 나타낼 수 있습니다.

In [ ]:
import albumentations as A

# crumpling 효과를 위한 변환 파이프라인
crumple_transform = A.Compose([
    A.GridDistortion(num_steps=5, distort_limit=0.3, p=1),  # GridDistortion으로 구김 효과 표현
    A.ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03, p=0.5)  # 추가적인 왜곡 효과
], keypoint_params=A.KeypointParams(format='xy'))

# 변환 적용 예시
augmented = crumple_transform(image=image, keypoints=bboxes.reshape(-1, 2).tolist())
augmented_image = augmented['image']
augmented_bboxes = np.array(augmented['keypoints']).reshape(-1, 8)